In [1]:
from langsmith import traceable
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")
if api_key is None:
    raise ValueError("Please set the GOOGLE_GENAI_API_KEY environment variable.")

os.environ["LANGSMITH_TRACING_V2"] = os.getenv("LANGSMITH_TRACING", "true")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")

llm = init_chat_model(
    "gemini-2.0-flash",
    model_provider="google_genai",
    api_key=api_key
  )


In [38]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
"""
Kamu adalah Teno, seorang konselor profesional yang penuh empati. Tugasmu adalah merespons dengan cara singkat, manusiawi, dan berfokus langsung pada inti masalah pengguna.
Gunakan bahasa sederhana, suportif, dan hindari kesan menggurui.

Informasi tambahan (jika ada): 
{context}

Pertanyaan dari pengguna:
{question}

Percakapan sebelumnya:
{history}

Berikan pemahaman atau dukungan yang hangat, lalu akhiri dengan kalimat yang membuka ruang agar pengguna mau berbagi lebih banyak atau memperdalam pembicaraan.
"""
)


In [3]:
from sentence_transformers import SentenceTransformer
from langchain.embeddings.base import Embeddings
import torch

# Load SBERT model dan pastikan menggunakan GPU jika tersedia
device = 'cuda' if torch.cuda.is_available() else 'cpu'
sbert_model = model = SentenceTransformer('naufalihsan/indonesian-sbert-large')
sbert_model = sbert_model.to(device)  # Pindahkan model ke GPU (jika ada)

# Custom embeddings class for SBERT
class SBERTEmbeddings(Embeddings):
    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        # Menggunakan model SBERT untuk menghasilkan embeddings
        embeddings = sbert_model.encode(texts, convert_to_tensor=True, show_progress_bar=True)
        embeddings = embeddings.to(device)  # Pindahkan embeddings ke GPU (jika ada)
        return embeddings.cpu().numpy().tolist()  # Pindahkan kembali ke CPU untuk konversi

    def embed_query(self, query: str) -> list[float]:
        # Menghasilkan embedding untuk query
        embedding = sbert_model.encode(query, convert_to_tensor=True)
        embedding = embedding.to(device)  # Pindahkan embedding ke GPU (jika ada)
        return embedding.cpu().numpy().tolist()  # Pindahkan kembali ke CPU untuk konversi

# Inisialisasi embeddings SBERT dan FAISS vector store
sbert_embeddings = SBERTEmbeddings()

d:\CODING\PYTHON\MACHINE_LEARNING\Tenangin\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\CODING\PYTHON\MACHINE_LEARNING\Tenangin\.venv\Lib\site-packages\transformers\utils\hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
def serialize_context(context_list):
    serialized = ""
    for idx, doc in enumerate(context_list, start=1):
        serialized += f"[Dokumen {idx}]\nJudul: {doc['BAB']}\nIsi: {doc['isi']}\n\n"
    return serialized


In [39]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
from langsmith import traceable

GREETINGS = ["halo", "assalamualaikum", "selamat malam", "apa kabar", "makasih", "terima kasih", "hai", "dadah", "hi"]
vector_store = FAISS.load_local("Embeddings_chonkie",sbert_embeddings,allow_dangerous_deserialization=True)


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str
    history : list[str]

def is_greeting(state: State) -> bool:
    return any(greeting in state["question"].lower() for greeting in GREETINGS)


def retrieve(state: State):
    if is_greeting(state):
        return {"context": []}  
    retrieved_docs = vector_store.similarity_search(state["question"],k=2)
    return {"context": retrieved_docs}

def generate(state: State):
    # Serialize context dari retrieval
    docs_content = [
        {
            "BAB": doc.metadata.get("bab", "Tidak diketahui"),
            "isi": doc.page_content
        }
        for doc in state["context"]
    ]
    serialized_context = serialize_context(docs_content)

    # Gabungkan history
    history_text = "\n".join(state.get("history", []))  # kalau kosong, default []

    # Siapin message yang lengkap: ada history + question sekarang
    messages = prompt.invoke({
        "history": history_text,
        "question": state["question"],
        "context": serialized_context
    })

    response = llm.invoke(messages)
    
    # Update history, nambahin pertanyaan baru ke list
    if(len(state.get("history", []))>3):
        state['history'].pop(0)
    updated_history = state.get("history", []) + [f"User: {state['question']}", f"AI: {response.content}"]

    return {
        "answer": response.content,
        "history": updated_history  # ⬅️ penting, biar state update!
    }



# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [22]:
vector_store.similarity_search("halo", k=2)

[Document(id='640933ae-1c45-4749-8223-8574e79d0967', metadata={'bab': 'BAB X NARKOTIKA, PSIKOTROPIKA DAN ZAT ADIKTIF'}, page_content='"sakaw" dari masing-masing obat berbeda tergantung dari jenis obatnya.(https://zulliesikawati.wordpress.com/?s=adiksi)\n10.2 Narkotika\nNarkotika berasal dari kata narkose (Yunani) yang artinya menidurkan atau membius, namun demikian narkotika bukan obat bius dan awalnya mengacu pada berbagai zat yang menumpulkan indra dan menghilangkan rasa sakit. Narkotika menekan saraf pusat sehingga menghasilkan efek penurunan sensitivitas terhadap rasa sakit dan aktivitas fisik, menimbulkan rasa kantuk.\nMenurut Undang-undang nomor 35 tahun 2009, narkotika adalah zat atau obat yang berasal dari tanaman atau bukan tanaman baik sintetis maupun semi sintetis yang dapat menyebabkan penurunan atau perubahan kesadaran, hilangnya rasa, mengurangi sampai menghilangkan rasa nyeri, dan dapat menimbulkan ketergantungan.\nNarkotika dapat berasal dari bahan alami seperti golonga

In [40]:
response = graph.invoke({"question": "halo"})
print(f"Anda    : {response['question']}")
print(f"Gemini  : {response['answer']}")
print(f"Context : {response['context']}")

Anda    : halo
Gemini  : Halo juga! Apa yang sedang kamu rasakan atau pikirkan saat ini? Ceritakan saja, aku di sini untuk mendengarkan.
Context : []
